In [34]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

#Constants:
L = 5000 #angstroms
T_test = 5730.9728 #Kelvin
P_test = 1.2594 
B = 0.1 #ratio of the #He/#H
A = 10**4 #ratio of #H/#metal
sigma_t = 6.655*10**(-25) #centimeter squared, used in thompson scattering 
m_h = 1.6726219*10**(-24) #grams, mass of ionized hydrogen, eg. a proton
c = 29979245800 #speed of light, centimeters per second
h = 6.62606896*10**(-27) #planck's constnat, erg * seconds
k = 1.38065*10**(-16) #boltzmann constant, ergs/Kelvin
Chi = 2.195*10**(-11) #ergs, ionization energy of hydrogen
x = 10000/L #substitution used in the absorption coefficients 
l = L/1000 #micrometers, wavelength of light, just converting L from angstroms to microns to clean up functions.
v = c/(L/10.**8) #frequency associated with L

check_ans = np.genfromtxt('opacities.dat', dtype='f8, names = ['temp', 'log_P', 'logH', 'logH-', 'logsigma'])
temp_pelog_dat = np.genfromtxt('temp_pelog.dat', dtype='f8', names = ['temp','log_P' ])

In [13]:
#Defining the Saha equation for hydrogen. Returns the ratio of n/n
def Saha_H(T, P):
    Saha_H_ans =  (5.0/2.0)*np.log10(T) - 13.595*(5040./T) - P - 0.4772
    return 10**Saha_H_ans

#Defining the Saha equation for metals. Returns the ratio of n/n
def Saha_metals(T, P):
    Saha_metals_ans = (5.0/2.0)*np.log10(T) - 7.9*(5040./T) - P - 0.0971
    return 10**Saha_metals_ans

print 'Saha_H_ans = ', Saha_H(T_test, P_test)
print 'Saha_metals_ans = ', Saha_metals(T_test, P_test)

#Defining theta
def Theta(T):
    theta = 5040./T
    return theta
H_ratio = Saha_H(T_test, P_test)
m_ratio = Saha_metals(T_test, P_test)

#Defining X = fraction of H ionized
def X(H_ratio):
    X_ans = H_ratio / (1 + H_ratio)
    return X_ans
print 'X = ', X(H_ratio)

#Defining Y = faction of metal ionized 
def Y(m_ratio):
    Y_ans = m_ratio / (1 + m_ratio)
    return Y_ans
print 'Y = ', Y(m_ratio)

Saha_H_ans =  5.04771580437e-05
Saha_metals_ans =  12.3469380576
X =  5.04746102288e-05
Y =  0.925076448569


In [15]:
#Defining Gaunt free-free factor:
def Gaunt_ff(T,l):
    Gaunt_ff_ans = 1.084 + 0.0188/Theta(T) + (0.00161 + 0.02661/Theta(T))*l - (0.0192 - 0.03889/Theta(T) + 0.02833/(Theta(T))**2 - 0.007828/(Theta(T))**3 + 0.0007304/(Theta(T))**4)*(l**2)
    return Gaunt_ff_ans
print 'Gaunt_ff_ans = ', Gaunt_ff(T_test, l)

#Defining coefficients for a, b, c for the Gaunt bound-free factor
m = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a_m = [0.9916, 1.105, 1.101, 0.9736, 1.03, 1.097, 1.098, 1., 1., 1.]
b_m = [0.09068, -0.7922, -0.329, 0., 0., 0., 0., 0., 0., 0.]
c_m = [-0.2524, 0.4536, 0.1152, 0., 0., 0., 0., 0, 0., 0.]
def Gaunt_bf(l,m):
    Gaunt_bf_ans = a_m[m-1] + b_m[m-1]*l + c_m[m-1]*(l**2)
    return Gaunt_bf_ans
print 'Gaunt_bf_ans for m=1 = ', Gaunt_bf(l, 1)

#Defining u, used in the absorption coefficient equation
def u(T, m):
    u_ans = (Chi/(k*T))/np.power(m, 2)
    return u_ans

#Finding m_o and m_star
test_m = (h * v) / (k*T_test)
for i in m:
    if test_m <= u(T_test,i):
        continue
    if test_m >= u(T_test, i):
        m_0 = i
        break
m_star =  m[9]

#Defining a new array of m's
new_m = np.arange(m_0, m_star + 1, 1)

#Doing the summation over new m's
summ = []
for i in new_m:
    factor = Gaunt_bf(l, i) * (np.exp(u(T_test, i))/(i**3))
    summ.append(factor)
summ = np.array(summ)
summ = np.sum(summ)

#Atomic hydrogen absorption coefficient per neutral hydrogren: alpha_lambda
eqn_part_2 = summ + (1/(2*u(T_test, 1)))*(np.exp(u(T_test, m_star)) - 1 + Gaunt_ff(T_test, l))
eqn_part_1 = (((2.0989*10**(-14))*np.exp(-u(T_test, 1)))/((l**3)*2))*abs(1-np.exp((-h*v)/k*T_test))
absorp_H = eqn_part_2 * eqn_part_1

#mass absorption coefficient per gram
multiplier = (1-X(H_ratio))/(1+4*B)*m_h
kappa_atomic = absorp_H*multiplier
print 'kappa_atomic = ', kappa_atomic

Gaunt_ff_ans =  1.23170234127
Gaunt_bf_ans for m=1 =  -4.865
kappa_atomic =  1.84094111469e-52


In [18]:
k_star = 0.00680133 + 0.178708*(l) + 0.164790*(l**2) - 0.024842*(l**3) + (l**4)*5.95244*10**(-4)

#Bound-free absorption coefficient
def absorp_bf(P,T):
    absorp_bf_ans = 10**(-26) * P * 0.4158*(Theta(T))**(5./2.) * np.exp(1.726*(Theta(T))) * abs(1-np.exp((-h*v)/k*T)) * k_star
    return absorp_bf_ans
#Free-free absorption coefficient
def absorp_ff(P,T):
    absorp_ff_ans = 10**(-26) * P * (0.0053666 - 0.011493*(Theta(T)) + 0.027029*(Theta(T))**(2) - (3.2062 - 11.924*(Theta(T)) + 5.939*(Theta(T))**(2))*(l/(10**6)) - (0.40192 - 7.0355*(Theta(T)) + 0.34592*(Theta(T))**(2))*((l**2)/(10**9)))
    return absorp_ff_ans

print 'absorp_bf_ans = ', absorp_bf(P_test, T_test)
print 'absorp_ff_ans = ', absorp_ff(P_test, T_test)


absorp_H_neg = absorp_bf(P_test, T_test) + absorp_ff(P_test, T_test)
kappa_ion = absorp_H_neg*multiplier
print 'kappa_ion = ', kappa_ion

absorp_bf_ans =  3.96287361893e-26
absorp_ff_ans =  2.03563478082e-28
kappa_ion =  4.75864385083e-50


In [29]:
#Calculating rayleigh scattering cross section, l is in angstroms
def cross_section_r(l):
    sigma_r_ans = (5.799*10**(-13))/(l**4) + 1.422*10**(-6)/(l**6) + 2.784/(l**8)
    return sigma_r_ans
print 'cross_section_r where l=5000Ang. = ', cross_section_r(5000)

wave_test = L*10**(-8) #Angstroms to centimeters
print cross_section_r(wave_test)
sigma_R_cm = cross_section_r(wave_test)*multiplier
sigma_R_ang = cross_section_r(5000)*multiplier
print 'sigma_R(cm) = ', sigma_R_cm
print 'sigma_R(angstroms) = ', sigma_R_ang

cross_section_r where l=5000Ang. =  1.02597504e-27
7.12704e+34
sigma_R(cm) =  85144582044.7
sigma_R(angstroms) =  1.22570121634e-51


In [30]:
#Calculating Thompson scattering
sigma_T = sigma_t * (X(H_ratio) + Y(m_ratio)/A)/((1.+4.*B)*m_h)
print 'sigma_T = ', sigma_T

sigma_total = sigma_R_ang + sigma_T
print 'sigma_total = ', sigma_total

sigma_T =  4.06353858316e-05
sigma_total =  4.06353858316e-05
